In [1]:
import blosc2
import numpy as np
import plotly.express as px
import ipywidgets as widgets

In [2]:
path = '/home/blosc/gaia/gaia-3d-windows-int8-3.b2nd'
# path = '/Users/martaiborra/gaia_plots/gaia-3d-windows-int8-3.b2nd'

In [3]:
arr = blosc2.open(path)
arr.info

type,NDArray
shape,"(20000, 20000, 20000)"
chunks,"(250, 250, 250)"
blocks,"(25, 25, 25)"
dtype,uint8
cratio,1140.70
cparams,"{'codec': , 'codec_meta': 0, 'clevel': 1, 'use_dict': 0, 'typesize': 1, 'nthreads': 16, 'blocksize': 15625, 'splitmode': , 'filters': [, , , , , ], 'filters_meta': [0, 0, 0, 0, 0, 0]}"
dparams,{'nthreads': 16}


In [11]:
shape = arr.shape
cube_shape = arr.chunks
cube_shape = np.array([100, 100, 100])
nsegments = 10

# global plot_output

plot_output = widgets.Output()


x_origin = widgets.IntText(
    min=0,
    max=shape[0] - cube_shape[0],
    step=1,
    description='X Origin:',
    value=0
)
y_origin = widgets.IntText(
    min=0,
    max=shape[1] - cube_shape[1],
    step=1,
    description='Y Origin:',
    value=0
)
z_origin = widgets.IntText(
    min=0,
    max=shape[2] - cube_shape[2],
    step=1,
    description='Z Origin:',
    value=0
)

In [12]:
axis_widget = widgets.Dropdown(
    options=[('x', 0), ('y', 1), ('z', 2)],
    value=0,
    description='Travel axis:',
    disabled=False,
)

In [13]:
def init(obj):
    global axis
    global step
    global start
    global nsteps
    global stop
    axis = axis_widget.value
    step = cube_shape[axis] // nsegments
    start = np.array([x_origin.value, y_origin.value, z_origin.value]) 
    nsteps = (shape[axis] - start[axis]) // step
    stop = start + np.array(cube_shape)
    # print("start ", start)
    # print("stop ", stop)

In [14]:
def frame_func(obj):
    obj.new
    npa = arr[start[0]:stop[0], start[1]:stop[1], start[2]:stop[2]]
    # print(stop)
    # print(step)
    # print(nsteps)
    # print(axis)
    img = np.sum(npa, axis=axis)     
    idx = np.indices(img.shape)
    fig = px.scatter(x=idx[0, :, :].flatten(), y=idx[1, :, :].flatten(), 
                     size=img.flatten(), opacity=0.8, 
                     color_discrete_sequence=['white']
                    )
    fig.update_traces(marker=dict(symbol="star"))
    fig.update_layout(plot_bgcolor="black", paper_bgcolor='white')
    
    plot_output.clear_output()
    with plot_output:
        fig.show()
    
    start[axis] += step
    stop[axis] += step


In [21]:
slider = widgets.IntSlider(
    min=0,
    max=100,# 10_000 - 250,
    step=25,
    description='Travel axis origin:',
    value=0
)

In [19]:
display(axis_widget)
coordinates_widgets = widgets.HBox(
[x_origin, y_origin, z_origin])
display(coordinates_widgets)

axis_widget.observe(init, names=['value'])
x_origin.observe(init, names=['value'])
y_origin.observe(init, names=['value'])
z_origin.observe(init, names=['value'])

a = 0
init(a)
display(slider)
slider.observe(frame_func, names=['value'])

Dropdown(description='Travel axis:', options=(('x', 0), ('y', 1), ('z', 2)), value=0)

IntSlider(value=0, description='Travel axis origin:', step=25)

In [20]:
display(plot_output)

Output()